In [ ]:
# ============================================
# BRAIN TUMOR CLASSIFICATION & ANOMALY DETECTION
# 02_model_training.ipynb
# ============================================

# Part 1: SUPERVISED LEARNING (VGG16)
# Binary Classification: Tumor vs No Tumor
# ============================================

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("=" * 60)
print("SECTION 1: SUPERVISED CNN WITH TRANSFER LEARNING")
print("=" * 60)

# Create models directory
os.makedirs("../models", exist_ok=True)

# Load data
train_dir = "../data/Training"
test_dir = "../data/Testing"

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

print(f"\n Training samples: {train_data.samples}")
print(f" Test samples: {test_data.samples}")
print(f" Classes: {train_data.class_indices}")

# Load pretrained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False

# Add custom layers
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
], name='Tumor_Classifier')

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train model
print("\n Training supervised classifier...")
history = model.fit(
    train_data, 
    validation_data=test_data, 
    epochs=10,
    verbose=1
)

# Plot loss over epochs
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend()
plt.title('Supervised Model Loss', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Plot accuracy over epochs
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend()
plt.title('Supervised Model Accuracy', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../models/supervised_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Save model
model.save("../models/tumor_classifier_vgg16.keras")
print("\n Supervised model saved: tumor_classifier_vgg16.keras")
print(f" Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")

# ============================================
# SECTION 2: UNSUPERVISED LEARNING (AUTOENCODER)
# Anomaly Detection: Trained only on healthy images
# ============================================

print("\n" + "=" * 60)
print("SECTION 2: UNSUPERVISED AUTOENCODER FOR ANOMALY DETECTION")
print("=" * 60)

# Data augmentation for autoencoder
train_datagen_ae = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load ONLY healthy images (no_tumor) for unsupervised learning
train_data_healthy = train_datagen_ae.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='input',  # Autoencoder: output = input
    classes=['no_tumor'],  # ONLY healthy brains!
    shuffle=True
)

print(f"\n Healthy training samples: {train_data_healthy.samples}")

# Build Convolutional Autoencoder
def build_autoencoder(input_shape=(150, 150, 3)):
    """Convolutional Autoencoder for image reconstruction"""
    
    input_img = layers.Input(shape=input_shape, name='input')
    
    # ENCODER
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 75x75
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 37x37
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)  # 19x19
    
    # DECODER
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D((2, 2))(x)  # 38x38
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D((2, 2))(x)  # 76x76
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D((2, 2))(x)  # 152x152
    
    # Crop to 150x150 and reconstruct RGB
    x = layers.Cropping2D(cropping=((1, 1), (1, 1)))(x)
    decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same', name='output')(x)
    
    return models.Model(input_img, decoded, name='Autoencoder')

# Build and compile
autoencoder = build_autoencoder()
autoencoder.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

autoencoder.summary()

# Callbacks
callbacks_ae = [
    EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
]

# Train autoencoder
print("\n Training autoencoder on healthy images only...")
history_ae = autoencoder.fit(
    train_data_healthy,
    epochs=50,
    steps_per_epoch=len(train_data_healthy),
    callbacks=callbacks_ae,
    verbose=1
)

# Plot training progress
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_ae.history['loss'], label='Training Loss (MSE)', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss (MSE)', fontsize=12)
plt.title('Autoencoder Training Loss', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_ae.history['mae'], label='Training MAE', linewidth=2, color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('MAE', fontsize=12)
plt.title('Mean Absolute Error', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../models/autoencoder_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Final training loss: {history_ae.history['loss'][-1]:.6f}")

# Test reconstruction quality
train_data_healthy.reset()
sample_images, _ = next(train_data_healthy)
reconstructed = autoencoder.predict(sample_images[:5], verbose=0)

plt.figure(figsize=(15, 6))
for i in range(5):
    # Original
    plt.subplot(2, 5, i + 1)
    plt.imshow(sample_images[i])
    plt.title('Original', fontsize=10)
    plt.axis('off')
    
    # Reconstructed
    plt.subplot(2, 5, i + 6)
    plt.imshow(reconstructed[i])
    error = np.mean(np.square(sample_images[i] - reconstructed[i]))
    plt.title(f'Reconstructed\nMSE: {error:.4f}', fontsize=10)
    plt.axis('off')

plt.suptitle('Reconstruction Quality on Healthy Brains', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../models/reconstruction_samples.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate threshold for anomaly detection
print("\n Calculating optimal threshold...")

train_data_healthy.reset()
all_healthy = []
for i in range(len(train_data_healthy)):
    batch, _ = next(train_data_healthy)
    all_healthy.append(batch)
all_healthy = np.concatenate(all_healthy)

reconstructed_healthy = autoencoder.predict(all_healthy, verbose=0)
errors_healthy = np.mean(np.square(all_healthy - reconstructed_healthy), axis=(1, 2, 3))

mean_error = np.mean(errors_healthy)
std_error = np.std(errors_healthy)
threshold_95 = np.percentile(errors_healthy, 95)
threshold_99 = np.percentile(errors_healthy, 99)

print(f"\n Reconstruction Error Statistics (healthy images):")
print(f"   Mean: {mean_error:.6f}")
print(f"   Std:  {std_error:.6f}")
print(f"   95th percentile: {threshold_95:.6f}")
print(f"   99th percentile: {threshold_99:.6f}")

# Save threshold
np.save('../models/anomaly_threshold.npy', threshold_95)
print(f"\n Threshold saved: {threshold_95:.6f}")

# Plot error distribution
plt.figure(figsize=(10, 6))
plt.hist(errors_healthy, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(threshold_95, color='red', linestyle='--', linewidth=2, 
            label=f'95th percentile: {threshold_95:.4f}')
plt.axvline(mean_error, color='green', linestyle='--', linewidth=2, 
            label=f'Mean: {mean_error:.4f}')
plt.xlabel('Reconstruction Error (MSE)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Error Distribution on Healthy Images', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../models/error_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Save autoencoder
autoencoder.save("../models/autoencoder_abnormality.keras")
print("\n Autoencoder saved: autoencoder_abnormality.keras")

print("\n" + "=" * 60)
print(" TRAINING COMPLETE!")
print("=" * 60)
print("\nModels created:")
print("  1. tumor_classifier_vgg16.keras (Supervised)")
print("  2. autoencoder_abnormality.keras (Unsupervised)")
print("  3. anomaly_threshold.npy (Threshold for detection)")
print("\n Next step: Run 03_evaluation.ipynb")


SECTION 1: SUPERVISED CNN WITH TRANSFER LEARNING
Found 2870 images belonging to 2 classes.
Found 394 images belonging to 2 classes.

 Training samples: 2870
 Test samples: 394
 Classes: {'no_tumor': 0, 'tumor': 1}



Model: "Tumor_Classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                              